In [17]:
import math, time, random, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import pickle
import warnings
warnings.filterwarnings('ignore')

In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [19]:
train = pd.read_csv("UNSW_NB15_training-set.csv")
test = pd.read_csv("UNSW_NB15_testing-set.csv")

In [20]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
data = pd.concat([train,test]).reset_index(drop = True)
cols_cat = data.select_dtypes("object").columns
cols_numeric = data._get_numeric_data().columns

In [21]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
data["service"].unique() # get rid of "-"
data["service"] = np.where(data["service"]=='-', "None", data["service"])
print(data["service"].unique())

['None' 'http' 'ftp' 'ftp-data' 'smtp' 'pop3' 'dns' 'snmp' 'ssl' 'dhcp'
 'irc' 'radius' 'ssh']


In [22]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
data = pd.get_dummies(data, columns = ["proto", "service", "state", "attack_cat"], dtype = "float64") 

In [23]:
data.drop(["attack_cat_Analysis", "attack_cat_Backdoor", "attack_cat_Exploits",
            "attack_cat_Fuzzers", "attack_cat_Reconnaissance", "attack_cat_Shellcode",
            "attack_cat_Worms", "attack_cat_Generic"], axis = 1, inplace = True)

In [24]:
data.head()

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,proto_3pc,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_aris,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_br-sat-mon,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cphb,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddp,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_emcon,proto_encap,proto_etherip,proto_fc,proto_fire,proto_ggp,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_iatp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_idrp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ip,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ipnip,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-opts,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-ip,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-1,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_micp,proto_mobile,proto_mtp,proto_mux,proto_narp,proto_netblt,proto_nsfnet-igp,proto_nvp,proto_ospf,proto_pgm,proto_pim,proto_pipe,proto_pnni,proto_pri-enc,proto_prm,proto_ptp,proto_pup,proto_pvp,proto_qnx,proto_rdp,proto_rsvp,proto_rtp,proto_rvd,proto_sat-expak,proto_sat-mon,proto_sccopmce,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_sep,proto_skip,proto_sm,proto_smp,proto_snp,proto_sprite-rpc,proto_sps,proto_srp,proto_st2,proto_stp,proto_sun-nd,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_tp++,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_None,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no,attack_cat_DoS,attack_cat_Normal
0,1,0.000011,2,0,496,0,90909.0902,254,0,180363632.0,0.0,0,0,0.011,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,0.000008,2,0,1762,0,125000.0003,254,0,881000000.0,0.0,0,0,0.008,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,0.000005,2,0,1068,0,200000.0051,254,0,854400000.0,0.0,0,0,0.005,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [25]:

data_drop = data.drop(["id", "attack_cat_DoS", "attack_cat_Normal"], axis = 1)

#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
cols_numeric = list(cols_numeric)
cols_numeric.remove("label")
cols_numeric.remove("id")

In [26]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
data_drop[cols_numeric] = (data_drop[cols_numeric] - np.min(data_drop[cols_numeric])) / np.std(data_drop[cols_numeric])

In [27]:
data_drop.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,proto_3pc,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_aris,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_br-sat-mon,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cphb,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddp,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_emcon,proto_encap,proto_etherip,proto_fc,proto_fire,proto_ggp,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_iatp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_idrp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ip,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ipnip,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-opts,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-ip,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-1,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_micp,proto_mobile,proto_mtp,proto_mux,proto_narp,proto_netblt,proto_nsfnet-igp,proto_nvp,proto_ospf,proto_pgm,proto_pim,proto_pipe,proto_pnni,proto_pri-enc,proto_prm,proto_ptp,proto_pup,proto_pvp,proto_qnx,proto_rdp,proto_rsvp,proto_rtp,proto_rvd,proto_sat-expak,proto_sat-mon,proto_sccopmce,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_sep,proto_skip,proto_sm,proto_smp,proto_snp,proto_sprite-rpc,proto_sps,proto_srp,proto_st2,proto_stp,proto_sun-nd,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_tp++,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_None,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no
0,1.841222e-06,0.007356,0.0,0.002716,0.0,0.566962,2.478337,0.0,0.971102,0.0,0.0,0.0,1.589104e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.087903,0.0,0.0,0.0,0.092339,2.015524,0.000000,0.000000,0.0,0.089922,0.0,0.0,0.0,0.000000,0.091956,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.339070e-06,0.007356,0.0,0.010002,0.0,0.779572,2.478337,0.0,4.743422,0.0,0.0,0.0,1.155712e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.162200,0.0,0.0,0.0,0.092339,2.015524,0.000000,0.000000,0.0,0.089922,0.0,0.0,0.0,0.000000,0.091956,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8.369190e-07,0.007356,0.0,0.006008,0.0,1.247316,2.478337,0.0,4.600204,0.0,0

In [28]:
data_drop.drop(["dur", "rate", "sloss", "djit", "tcprtt", "trans_depth", "ct_src_dport_ltm", "ct_flw_http_mthd",
                      "proto_3pc", "proto_aris", "proto_br-sat-mon", "proto_cphb", "proto_ddp", "proto_emcon",
                      "proto_ggp", "proto_iatp", "proto_idrp", "proto_ip", "proto_ipnip", "proto_ipv6-opts", 
                      "proto_iso-ip", "proto_leaf-1", "proto_micp", "proto_netblt", "proto_pim", "proto_ptp",
                      "proto_rsvp", "proto_sccopmce", "proto_sep", "proto_sprite-rpc", "proto_sun-nd", "proto_tp++",
                      "proto_unas", "proto_vrrp", "proto_xns-idp", "service_dns", "service_pop3", "service_ssl", "state_FIN",
                      "state_URN"], axis = 1, inplace = True)

In [29]:
data_drop.head()

,spkts,dpkts,sbytes,dbytes,sttl,dttl,sload,dload,dloss,sinpkt,dinpkt,sjit,swin,stcpb,dtcpb,dwin,synack,ackdat,smean,dmean,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_encap,proto_etherip,proto_fc,proto_fire,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_mobile,proto_mtp,proto_mux,proto_narp,proto_nsfnet-igp,proto_nvp,proto_ospf,proto_pgm,proto_pipe,proto_pnni,proto_pri-enc,proto_prm,proto_pup,proto_pvp,proto_qnx,proto_rdp,proto_rtp,proto_rvd,proto_sat-expak,proto_sat-mon,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_skip,proto_sm,proto_smp,proto_snp,proto_sps,proto_srp,proto_st2,proto_stp,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xtp,proto_zero,service_None,service_dhcp,service_ftp,service_ftp-data,service_http,service_irc,service_radius,service_smtp,service_snmp,service_ssh,state_ACC,state_CLO,state_CON,state_ECO,state_INT,state_PAR,state_REQ,state_RST,state_no
0,0.007356,0.0,0.002716,0.0,2.478337,0.0,0.971102,0.0,0.0,1.589104e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.087903,0.0,0.0,0.092339,2.015524,0.000000,0.0,0.089922,0.0,0.0,0.000000,0.091956,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.007356,0.0,0.010002,0.0,2.478337,0.0,4.743422,0.0,0.0,1.155712e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.162200,0.0,0.0,0.092339,2.015524,0.000000,0.0,0.089922,0.0,0.0,0.000000,0.091956,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.007356,0.0,0.006008,0.0,2.478337,0.0,4.600204,0.0,0.0,7.223200e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.476922,0.0,0.0,0.184678,2.015524,0.000000,0.0,0.179844,0.0,0.0,0.000000,0.183913,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.007356,0.0,0.005041,0.0,2.478337,0.0,3.230480,0.0,0.0,8.667840e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.068958,0.0,0.0,0.184678,2.015524,0.122343,0.0,0.179844,0.0,0.0,0.119101,0.183913,0.0,0,0.0,

In [30]:
rf_features = data_drop[["label", "ct_state_ttl", "sload", "sttl", "dload", "sbytes",
                         "dttl", "dinpkt", "smean", "synack", "ct_srv_dst", "dbytes",
                         "sinpkt", "ct_dst_ltm", "ackdat", "dmean", "dpkts", "ct_srv_src",
                         "sjit", "state_INT", "ct_src_ltm","ct_dst_ltm", "spkts", "ct_dst_sport_ltm",
                         "stcpb", "dloss"]]

In [31]:
print(rf_features.shape)
rf_features.head()

(257673, 26)


,label,ct_state_ttl,sload,sttl,dload,sbytes,dttl,dinpkt,smean,synack,ct_srv_dst,dbytes,sinpkt,ct_dst_ltm,ackdat,dmean,dpkts,ct_srv_src,sjit,state_INT,ct_src_ltm,ct_dst_ltm,spkts,ct_dst_sport_ltm,stcpb,dloss
0,0,2.015524,0.971102,2.478337,0.0,0.002716,0.0,0.0,1.087903,0.0,0.091956,0.0,1.589104e-06,0.000000,0.0,0.0,0.0,0.092339,0.0,1.0,0.000000,0.000000,0.007356,0.0,0.0,0.0
1,0,2.015524,4.743422,2.478337,0.0,0.010002,0.0,0.0,4.162200,0.0,0.091956,0.0,1.155712e-06,0.000000,0.0,0.0,0.0,0.092339,0.0,1.0,0.000000,0.000000,0.007356,0.0,0.0,0.0
2,0,2.015524,4.600204,2.478337,0.0,0.006008,0.0,0.0,2.476922,0.0,0.183913,0.0,7.223200e-07,0.000000,0.0,0.0,0.0,0.184678,0.0,1.0,0.000000,0.000000,0.007356,0.0,0.0,0.0
3,0,2.015524,3.230480,2.478337,0.0,0.005041,0.0,0.0,2.068958,0.0,0.183913,0.0,8.667840e-07,0.122343,0.0,0.0,0.0,0.184678,0.0,1.0,0.119101,0.122343,0.007356,0.0,0.0,0.0
4,0,2.015524,4.578667,2.478337,0.0,0.012096,0.0,0.0,5.046121,0.0,0.183913,0.0,1.444640e-06,0.122343,0.0,0.0,0.0,0.184678,0.0,1.0,0.119101,0.122343,0.007356,0.0,0.0,0.0


In [32]:
from sklearn import model_selection
from sklearn import metrics
X = rf_features.drop("label", axis = 1)
Y = rf_features["label"]

In [33]:
global X

In [34]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
def fit_algo(algo, x, y, cv):
    model = algo.fit(x, y)
    
    #check its score
    acc = round(model.score(x, y) *100, 2)
    y_pred = model_selection.cross_val_predict(algo, x, y, cv = cv,
                                              n_jobs = -1)
    
    acc_cv = round(metrics.accuracy_score(Y,y_pred)*100,2)
    
    return y_pred, acc, acc_cv, model

In [35]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.linear_model import LogisticRegression
start_time = time.time()

pred_now, acc_lr, acc_cv_lr, lr = fit_algo(LogisticRegression(C = 0.1), X, Y, 10)

lr_time = (time.time() - start_time)

print("Accuracy: %s" % acc_lr)
print("Accuracy of CV: %s" % acc_cv_lr)
print("Execution time: %s" % lr_time)

Accuracy: 88.22
Accuracy of CV: 87.54
Execution time: 37.972779273986816


In [36]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.neighbors import KNeighborsClassifier
start_time = time.time()
pred_now, acc_knn, acc_cv_knn, knn = fit_algo(KNeighborsClassifier(n_neighbors = 3)
                                        , X, Y, 10)
knn_time = (time.time() - start_time)
print("Accuracy: %s" % acc_knn)
print("Accuracy of CV: %s" % acc_cv_knn)
print("Execution time: %s" % knn_time)

Accuracy: 95.62
Accuracy of CV: 89.22
Execution time: 834.1954407691956


In [37]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
start_time = time.time()
pred_now, acc_dt, acc_cv_dt, dt = fit_algo(DecisionTreeClassifier(random_state = 1)
                                        , X, Y, 10)

dt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_dt)
print("Accuracy of CV: %s" % acc_cv_dt)
print("Execution time: %s" % dt_time)




Accuracy: 99.68
Accuracy of CV: 90.97
Execution time: 24.885576486587524


In [38]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Random Forest
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()
pred_now, acc_rf, acc_cv_rf, rf = fit_algo(RandomForestClassifier(n_estimators = 100)
                                        , X, Y, 10)

rf_time = (time.time() - start_time)

print("Accuracy: %s" % acc_rf)
print("Accuracy of CV: %s" % acc_cv_rf)
print("Execution time: %s" % rf_time)

Accuracy: 99.68
Accuracy of CV: 92.02
Execution time: 399.53236746788025


In [39]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gaussian Naive Bayes 
from sklearn.naive_bayes import GaussianNB
start_time = time.time()

pred_now, acc_gnb, acc_cv_gnb, gnb= fit_algo(GaussianNB()
                                        ,X,Y,5)

gnb_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gnb)
print("Accuracy of CV: %s" % acc_cv_gnb)
print("Execution time: %s" % gnb_time)

Accuracy: 75.88
Accuracy of CV: 75.95
Execution time: 1.1981890201568604


In [40]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
start_time = time.time()

pred_now, acc_gbt, acc_cv_gbt, gbt= fit_algo(GradientBoostingClassifier()
                                        , X, Y, 10)

gbt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gbt)
print("Accuracy of CV: %s" % acc_cv_gbt)
print("Execution time: %s" % gbt_time)



Accuracy: 92.68
Accuracy of CV: 91.32
Execution time: 826.806227684021


In [41]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Cross Validation Accuracy Comparison
algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'CV Accuracy %' : [acc_cv_lr, acc_cv_dt, acc_cv_rf, acc_cv_gnb, acc_cv_gbt, acc_cv_knn] })
acc_df = acc_df.sort_values(by='CV Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df

,Algorithm,CV Accuracy %
0,RandomForest,92.02
1,GBC,91.32
2,Decision Tree,90.97
3,KNN,89.22
4,Log. Reg.,87.54
5,Gaussian NB,75.95


In [42]:
# Accuracy Comparison

algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'Accuracy %' : [acc_lr, acc_dt, acc_rf, acc_gnb, acc_gbt, acc_knn] })
acc_df = acc_df.sort_values(by='Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df

,Algorithm,Accuracy %
0,Decision Tree,99.68
1,RandomForest,99.68
2,KNN,95.62
3,GBC,92.68
4,Log. Reg.,88.22
5,Gaussian NB,75.88


In [43]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
fimp_rf = pd.DataFrame({'Feature' : X.columns, 'Importance' : (rf.feature_importances_).astype(float)})
fimp_rf = fimp_rf.sort_values(by='Importance', ascending=False)
fimp_rf.head(37)

,Feature,Importance
2,sttl,0.184078
0,ct_state_ttl,0.132480
3,dload,0.070340
1,sload,0.067643
4,sbytes,0.059520
7,smean,0.051875
9,ct_srv_dst,0.050054
14,dmean,0.040127
11,sinpkt,0.038400
16,ct_srv_src,0.036386


50%

In [44]:
x = data_drop.copy()

In [45]:
from sklearn.model_selection import train_test_split
SEED = 1

x_train, x_test = train_test_split(x, train_size = 0.50, random_state = SEED)

In [46]:
print(x_train.shape)
x_train.head()

(128836, 157)


,spkts,dpkts,sbytes,dbytes,sttl,dttl,sload,dload,dloss,sinpkt,dinpkt,sjit,swin,stcpb,dtcpb,dwin,synack,ackdat,smean,dmean,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_encap,proto_etherip,proto_fc,proto_fire,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_mobile,proto_mtp,proto_mux,proto_narp,proto_nsfnet-igp,proto_nvp,proto_ospf,proto_pgm,proto_pipe,proto_pnni,proto_pri-enc,proto_prm,proto_pup,proto_pvp,proto_qnx,proto_rdp,proto_rtp,proto_rvd,proto_sat-expak,proto_sat-mon,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_skip,proto_sm,proto_smp,proto_snp,proto_sps,proto_srp,proto_st2,proto_stp,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xtp,proto_zero,service_None,service_dhcp,service_ftp,service_ftp-data,service_http,service_irc,service_radius,service_smtp,service_snmp,service_ssh,state_ACC,state_CLO,state_CON,state_ECO,state_INT,state_PAR,state_REQ,state_RST,state_no
115986,0.007356,0.017859,0.000702,0.001218,0.302474,0.257179,0.003157,0.296331,0.0,1.444640e-06,0.000005,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.237979,0.350338,0.0,0.000000,0.000000,0.122343,0.000000,0.089922,0.0,0.0,0.476403,0.183913,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
201332,0.007356,0.000000,0.000518,0.000000,2.478337,0.000000,0.613791,0.000000,0.0,5.778560e-07,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.160271,0.000000,0.0,1.385082,2.015524,1.835147,2.572235,1.348832,0.0,0.0,1.905613,1.379344,0.000000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
192002,0.007356,0.000000,0.001013,0.000000,2.478337,0.000000,0.478590,0.000000,0.0,1.300176e-06,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.369110,0.000000,0.0,0.369355,2.015524,0.244686,0.342965,0.359689,0.0,0.0,0.238202,0.367825,0.000000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
80789,0.007356,0.000000,0.000380,0.000000,2.47

In [47]:
from sklearn import model_selection
from sklearn import metrics
X = x_train.drop("label", axis = 1)
Y = x_train["label"]

In [48]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.linear_model import LogisticRegression
start_time = time.time()

pred_now, acc_lr, acc_cv_lr, lr = fit_algo(LogisticRegression(C = 0.1), X, Y, 10)

lr_time = (time.time() - start_time)

print("Accuracy: %s" % acc_lr)
print("Accuracy of CV: %s" % acc_cv_lr)
print("Execution time: %s" % lr_time)

Accuracy: 89.89
Accuracy of CV: 89.84
Execution time: 42.674638986587524


In [49]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.neighbors import KNeighborsClassifier
start_time = time.time()
pred_now, acc_knn, acc_cv_knn, knn = fit_algo(KNeighborsClassifier(n_neighbors = 3)
                                        , X, Y, 10)
knn_time = (time.time() - start_time)
print("Accuracy: %s" % acc_knn)
print("Accuracy of CV: %s" % acc_cv_knn)
print("Execution time: %s" % knn_time)

Accuracy: 95.62
Accuracy of CV: 91.17
Execution time: 1004.976286649704


In [50]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Decision Tree


from sklearn.tree import DecisionTreeClassifier
start_time = time.time()
pred_now, acc_dt, acc_cv_dt, dt = fit_algo(DecisionTreeClassifier(random_state = 1)
                                        , X, Y, 10)

dt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_dt)
print("Accuracy of CV: %s" % acc_cv_dt)
print("Execution time: %s" % dt_time)




Accuracy: 99.8
Accuracy of CV: 93.25
Execution time: 21.208158254623413


In [51]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Random Forest 
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()
pred_now, acc_rf, acc_cv_rf, rf = fit_algo(RandomForestClassifier(n_estimators = 100)
                                        , X, Y, 10)

rf_time = (time.time() - start_time)

print("Accuracy: %s" % acc_rf)
print("Accuracy of CV: %s" % acc_cv_rf)
print("Execution time: %s" % rf_time)

Accuracy: 99.8
Accuracy of CV: 94.68
Execution time: 189.78665256500244


In [52]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gaussian Naive Bayes 
from sklearn.naive_bayes import GaussianNB
start_time = time.time()

pred_now, acc_gnb, acc_cv_gnb, gnb= fit_algo(GaussianNB()
                                        ,X,Y,5)

gnb_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gnb)
print("Accuracy of CV: %s" % acc_cv_gnb)
print("Execution time: %s" % gnb_time)

Accuracy: 42.17
Accuracy of CV: 42.17
Execution time: 2.457131862640381


In [53]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gradient Boosting


from sklearn.ensemble import GradientBoostingClassifier
start_time = time.time()

pred_now, acc_gbt, acc_cv_gbt, gbt= fit_algo(GradientBoostingClassifier()
                                        , X, Y, 10)

gbt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gbt)
print("Accuracy of CV: %s" % acc_cv_gbt)
print("Execution time: %s" % gbt_time)



Accuracy: 93.43
Accuracy of CV: 93.33
Execution time: 665.1068453788757


In [54]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Cross Validation Accuracy Comparison


algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'CV Accuracy %' : [acc_cv_lr, acc_cv_dt, acc_cv_rf, acc_cv_gnb, acc_cv_gbt, acc_cv_knn] })
acc_df = acc_df.sort_values(by='CV Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df


,Algorithm,CV Accuracy %
0,RandomForest,94.68
1,GBC,93.33
2,Decision Tree,93.25
3,KNN,91.17
4,Log. Reg.,89.84
5,Gaussian NB,42.17


In [55]:

# Accuracy Comparison

algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'Accuracy %' : [acc_lr, acc_dt, acc_rf, acc_gnb, acc_gbt, acc_knn] })
acc_df = acc_df.sort_values(by='Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df

,Algorithm,Accuracy %
0,Decision Tree,99.80
1,RandomForest,99.80
2,KNN,95.62
3,GBC,93.43
4,Log. Reg.,89.89
5,Gaussian NB,42.17


In [56]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda

fimp_rf = pd.DataFrame({'Feature' : X.columns, 'Importance' : (rf.feature_importances_).astype(float)})
fimp_rf = fimp_rf.sort_values(by='Importance', ascending=False)
fimp_rf.head(10)



,Feature,Importance
4,sttl,0.121206
22,ct_state_ttl,0.082166
6,sload,0.057532
7,dload,0.052262
2,sbytes,0.049676
10,dinpkt,0.047616
18,smean,0.042376
19,dmean,0.041695
25,ct_dst_src_ltm,0.040628
16,synack,0.037318
